In [2]:
# Inspired by notebook from Aguiar
# Func: Extract all features, flatten json columns
# with multiprocessing module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys
import math
from pandas.io.json import json_normalize
from datetime import datetime
import os
import time
from datetime import datetime
from ast import literal_eval
import multiprocessing
import glob
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [5]:
%time train_df = pd.read_csv("./data/my_train.csv",low_memory=False)

CPU times: user 54.2 s, sys: 3.17 s, total: 57.3 s
Wall time: 57.3 s


In [7]:
%time test_df = pd.read_csv("./data/my_test.csv",low_memory=False)

CPU times: user 11.8 s, sys: 428 ms, total: 12.2 s
Wall time: 12.2 s


In [11]:
# Find columne that is constant
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False) < 2]
const_cols

['hits_customDimensions',
 'hits_customMetrics',
 'hits_customVariables',
 'hits_experiment',
 'hits_index',
 'hits_product_customDimensions',
 'hits_product_customMetrics',
 'hits_product_index',
 'hits_promotion_index',
 'hits_publisher_infos']

In [12]:
print(train_df.shape)
print(test_df.shape)

(1708345, 159)
(401590, 158)


In [14]:
# find columns difference
col1 = [c for c in train_df.columns if c not in test_df.columns]
col2 = [c for c in test_df.columns if c not in train_df.columns]
print(col1)
print(col2)

['trafficSource.campaignCode']
[]


In [16]:
# Drop empty columnes
def drop_cols(df,cols):
    for col in cols:
        if col in df.columns:
            df.drop(col,axis=1,inplace=True)
            
            
drop_cols(train_df,const_cols + ['trafficSource.campaignCode'])
drop_cols(test_df,const_cols)

In [17]:
print(train_df.shape)
print(test_df.shape)

(1708345, 148)
(401590, 148)


In [23]:
# Check the data in train and test
print("train date min max {0} {1}".format(
train_df['date'].min(),
train_df['date'].max()
))
print("test date min max {0} {1}".format(
test_df['date'].min(),
test_df['date'].max()
))


train date min max 20160801 No
test date min max 20180501 No


In [26]:
test_df['date'].unique()

array(['20181012', '20180505', '20180911', '20180709', '20180726',
       '20180526', '20180524', '20180713', '20180625', '20180506',
       '20180704', '20180909', '20180718', '20180531', '20180714',
       '20181003', '20180916', '20180529', '20180525', '20181006',
       '20180630', '20180904', '20180611', '20180823', '20180908',
       '20180629', '20180503', '20180824', '20180719', '20180928',
       '20180707', '20180612', '20180624', '20180725', '20180521',
       '20180511', '20180602', '20180819', '20180721', '20181004',
       '20180924', '20180921', '20180509', '20180801', '20180604',
       '20180818', '20180712', '20180507', '20180608', '20180519',
       '20180523', '20180614', '20181005', '20180813', '20180826',
       '20180618', '20181010', '20180706', '20180910', '20180703',
       '20180730', '20180728', '20180809', '20180926', '20180814',
       '20180616', '20180903', '20180925', '20180501', '20180907',
       '20180802', '20180609', '20181007', '20180906', '201809

In [25]:
train_df['date']

0          20161208
1          20161208
2          20161208
3          20161208
4          20161208
5          20161208
6          20161208
7          20161208
8          20161208
9          20161208
10         20161208
11         20161208
12         20161208
13         20161208
14         20161208
15         20161208
16         20161208
17         20161208
18         20161208
19         20161208
20         20161208
21         20161208
22         20161208
23         20161208
24         20161208
25         20161208
26         20161208
27         20161208
28         20161208
29         20161208
30         20161208
31         20161208
32         20161208
33         20161208
34         20161208
35         20161208
36         20161208
37         20161208
38         20161208
39         20161208
40         20161208
41         20161208
42         20161208
43         20161208
44         20161208
45         20161208
46         20161208
47         20161208
48         20161208
49         20161208


In [28]:
train_df.columns.values

array(['channelGrouping', 'customDimensions_index',
       'customDimensions_value', 'date', 'device.browser',
       'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'fullVisitorId',
       'geoNetwork.city', 'geoNetwork.cityId', 'geoNetwork.continent',
       'geoNetwork.country', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent',
       'hits_appInfo.exitScreenName', 'hits_appInfo.landingScreenName',
       'hits_appInfo.screenDepth', 'hits_appInfo.screenName',
 

In [33]:
for df in train_df,test_df:
    df['totals.transactionRevenue'].fillna(0,inplace=True)

In [36]:
test_df[test_df['totals.transactionRevenue']!=0]['totals.transactionRevenue']

143       4.159000e+07
167       7.999000e+07
240       1.759000e+07
241       3.518000e+07
261       1.299000e+07
282       4.392000e+07
292       7.275000e+07
298       5.498000e+07
301       8.593000e+07
311       5.199000e+07
314       8.694600e+08
320       6.399000e+07
328       4.747000e+07
341       5.892000e+07
342       4.235000e+07
344       1.299000e+07
347       7.396000e+07
354       8.793000e+07
394       4.398000e+07
2269      2.199000e+07
2330      4.159000e+07
2429      4.398000e+07
2452      2.399000e+07
2483      5.277000e+07
2484      1.120000e+08
2488      1.299000e+07
2491      4.159000e+07
2503      6.399000e+07
2526      3.838000e+07
2552      1.599000e+07
2557      4.790000e+08
2560      2.397000e+07
2563      3.677000e+07
2564      3.591000e+08
2569      5.999000e+07
2570      1.151700e+08
2578      2.199000e+07
2589      2.078000e+07
2592      7.976000e+07
2596      2.913000e+08
2599      1.887800e+08
2605      1.799250e+09
2610      3.798000e+07
2624      2